# Segmenting and Clustering Neighborhoods in Toronto

## Answer1 - To explore and cluster the neighborhoods in Toronto

### Importing Libraries

In [73]:
# Importing Libraries

import pandas as pd
import numpy as np
from urllib.request import urlopen
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
# Installing Geopy and Folium

!conda install lxml --yes

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim


!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [74]:
# To Obtain the data  that is in postal codes and to transform the data into panda dataframe
post_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# oepn url to get postal codes
html = urlopen(post_url)

In [75]:
# Using Pandas to parse the table on the above web page
post_table = pd.read_html(post_url, header =0)

df = post_table[0]

#dataframe with  three columns: PostalCode, Borough, and Neighborhood
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [76]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
df2 = df[df.Borough != 'Not assigned']

In [77]:
df3 = df2
df3.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [78]:
# Combining Neighborhood with comma
df4 = df3.pivot_table( index = ['Postal code','Borough'], values = 'Neighborhood', aggfunc=','.join)
df4.reset_index(inplace = True)
df4.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### There are no multiple entries for any  postal code in given dataset, so there will not be combining neighborhood with comma 
#### Please refer https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [80]:
df5  = df4

In [81]:
#Getting same neighbourhood as borough except "not assigned" value
for count, i in enumerate(df5['Neighborhood']):
    if(i=="Not assigned"):
        df5.loc[count]["Neighborhood"] = df5.loc[count]["Borough"]

In [82]:
df5.shape

(103, 3)

## Answer 2 - To Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name

### URL for coordinates

In [83]:
cood_url='http://cocl.us/Geospatial_data'

In [84]:
# reaind csv file
dfCood = pd.read_csv(cood_url)
dfCood.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [85]:
dfCood.shape

(103, 3)

### Merging the  Labels

In [86]:
result = pd.merge(df5, dfCood, left_on = ['Postal code'], right_on=['Postal Code'], how = 'left')


In [87]:
result.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1C,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### Removing extra column Postal Code

In [88]:
result.drop(columns= ['Postal Code'])

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


In [89]:
result.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,M1B,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,M1C,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [90]:
result.shape

(103, 6)

## Answer 3 - Explore and cluster the neighborhoods in Toronto - contains the word Toronto

In [91]:
address = 'Toronto, Canada'

### Getting coordinates for the address

In [92]:
geolocator = Nominatim(user_agent='city_explorer')
location = geolocator.geocode(address)
latitude = location.latitude

### Toronto Map

In [93]:
map_toronto = folium.Map(location = [latitude,longitude], zoom_start =10)
location_df = result
for lat, lng, Borough, Neighborhood in zip(location_df['Latitude'],location_df['Longitude'],location_df['Borough'],location_df['Neighborhood']):
    label = '{},{}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat,lng],
    radius =5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = 'red',
    fill_opacity =0.7,
    parse_html=False).add_to(map_toronto)

In [94]:
map_toronto

In [95]:
map_toronto.save('map_toronto.html')

In [96]:
#Define Foursquare Credentials and Version
CLIENT_ID = '2KUTZIDNIX1XXIMPSIGI3MPNZ1LIKBXBC055SHTFZOJRY5CK' # your Foursquare ID
CLIENT_SECRET = 'HM0D2S4YT5G5GKJDBWM0H4FUQCHY121OWXAX44CQNRAOO31M' # your Foursquare Secret
VERSION = '20200406' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2KUTZIDNIX1XXIMPSIGI3MPNZ1LIKBXBC055SHTFZOJRY5CK
CLIENT_SECRET:HM0D2S4YT5G5GKJDBWM0H4FUQCHY121OWXAX44CQNRAOO31M


## Exploring Neighbourhoods in toronto

In [97]:
LIMIT =100
def explore(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [98]:
toronto_venues = explore(names = result["Neighborhood"],
                        latitudes = result["Latitude"],
                        longitudes = result["Longitude"])


In [99]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Alderwood / Long Branch,11,11,11,11,11,11
Bathurst Manor / Wilson Heights / Downsview North,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,23,23,23,23,23,23
Berczy Park,56,56,56,56,56,56
Birch Cliff / Cliffside West,4,4,4,4,4,4
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Business reply mail Processing CentrE,17,17,17,17,17,17


In [100]:
## One hot encoding for venue categories
toronto_onehot_encoding = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot_encoding['Neighborhood'] = toronto_venues['Neighborhood']
fixed_columns = [toronto_onehot_encoding.columns[-1]] + list(toronto_onehot_encoding.columns[:-1])
toronto_onehot_encoding = toronto_onehot_encoding[fixed_columns]

In [101]:
toronto_grouped = toronto_onehot_encoding.groupby('Neighborhood').mean().reset_index()
toronto_grouped


,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,Alderwood / Long Branch,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,Bathurst Manor / Wilson Heights / Downsview North,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.00000
3,Bayview Village,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,Bedford Park / Lawrence Manor East,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
5,Berczy Park,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
6,Birch Cliff / Cliffside West,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
7,Brockton / Parkdale Village / Exhibition Place,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
8,Business reply mail Processing CentrE,0.058824,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
9,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.00000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [102]:
## Using only select top venues for location
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [103]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


## Clustering locations into 7 clusters using KMeans

In [104]:
kclusters = 7
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = result
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [105]:
## Checking for null values
toronto_merged.index[toronto_merged["Cluster Labels"].isnull() == True].tolist()

## Dropping null values
toronto_merged = toronto_merged.dropna()

In [106]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)

In [107]:
map_clusters

In [108]:
map_clusters.save('map_clusters.html')